In [2]:
%matplotlib inline

import pymysql
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import scale
from IPython.display import display, HTML

In [40]:
conn = pymysql.connect(host='104.196.5.55', user='root', passwd='', db='nfl')

cur = conn.cursor()

In [113]:
#QB data

#for each tbl in list of tbls:
#    query_sql(tbl_name)

#def query_sql(tbl_name):
#    strSQL = ("Select * from" + tbl_name)
#    print strSQL

#Create pass dataframe with passes 40 yards and over
strSQL = ("Select * from pass")
df_pass = pd.DataFrame(pd.read_sql(strSQL,conn,params=None))

#Create offense dataframe with all offense data
strSQL = ("Select * from offense")
df_offense = pd.DataFrame(pd.read_sql(strSQL,conn,params=None))

#Create offense dataframe with all offense data
strSQL = ("Select * from td")
df_td = pd.DataFrame(pd.read_sql(strSQL,conn,params=None))

#Create play dataframe with all offense data
strSQL = ("Select * from play")
df_play = pd.DataFrame(pd.read_sql(strSQL,conn,params=None))

In [92]:
def query_sql(tbl_name):
    global df
    strSQL = ("Select * from " + tbl_name)
    df = pd.DataFrame(pd.read_sql(strSQL,conn,params=None))
    df.name = "df_"+tbl_name
    print df.name
    
query_sql("fumble") 

df_fumble


In [114]:
#Grab all relevant tables for QB

#Passing yards
pass_yrds = df_all_offense_2015['py']/5*.2

#TD Passes
pass_tds = df_all_offense_2015['tdp']*4


#pass_two_pt =

#40+ yard td pass

#50+ yard td pass

#Int thrown
#pass_ints = df_all_offense_2015['ints']*-2

#2 pt conversions






In [139]:
#Inner join pass table and td table on play id to get passer
#Keep pid, yds_y, psr, trg
df_pass_td = pd.merge(df_td,df_pass,how='inner',on='pid')
#df_pass_td should have all plays that are tds
df_pass_td = df_pass_td[['pid','yds_y','psr','trg']]
df_pass_td_4050 = df_pass_td[df_pass_td['yds_y']>39]

#Merge new pass td table with play table to get gid
df_pass_td_gid = pd.merge(df_pass_td_4050,df_play,how='left',on='pid')
df_pass_td_gid = df_pass_td_gid[['pid','gid','yds_y','psr','trg']]

#Create binary for 40 yard and 50 yard td
df_pass_td_gid['40_yd_td'] = df_pass_td_gid['yds_y'].map(lambda x: 1 if 40 <= x < 50 else 0)
df_pass_td_gid['50_yd_td'] = df_pass_td_gid['yds_y'].map(lambda x: 1 if 50 <= x else 0)


In [149]:
#df_pass_td_gid[['gid','psr','trg']].groupby(['gid','psr','trg']).agg(['count'])
df_pass_td_gid

,pid,gid,yds_y,psr,trg,40_yd_td,50_yd_td
0,75,1,44,CC-1400,TM-0900,1,0
1,98,1,48,CC-1400,SJ-0400,1,0
2,1051,7,48,CM-2200,MR-1400,1,0
3,1819,12,53,TB-0200,QI-0100,0,1
4,2365,15,72,KW-0500,MF-0300,0,1
5,2379,15,80,KW-0500,AH-0200,0,1
6,2450,16,45,MB-4500,JS-4400,1,0
7,2465,16,43,MB-4500,JS-4400,1,0
8,2608,16,40,MB-4500,JS-4400,1,0
9,3414,21,50,PM-0200,MH-1200,0,1


In [24]:
def qb_points():
    #Passing points
    pass_yrds = df_all_offense_2015['py']/5*.2
    pass_40_tds = 
    pass_50_tds = 
    pass_tds = df_all_offense_2015['tdp']*4
    pass_ints = df_all_offense_2015['ints']*-2
    pass_two_pt =
    
    #Rushing points
    rush_yrds = 
    rush_tds = 
    rush_40_tds = 
    rush_50_tds = 
    rush_two_pt =
    
    #Receiving yards =
    
    
    